# Basic Import

In [1]:
import pandas as pd
import numpy as np
import tsfresh
import matplotlib.pyplot as plt
import os
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv1D, Dropout, MaxPooling1D, BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import metrics as kmetrics
import tensorflow.keras.backend as K

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


2022-10-16 00:47:45.067198: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-16 00:47:45.575764: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/jingyuan/catkin_ws/devel/lib:/opt/ros/noetic/lib
2022-10-16 00:47:45.575785: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-16 00:47:45.631407: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been regist

# Loading Data

In [2]:
base_df = pd.DataFrame()

folder_path = "/home/jingyuan/Documents/pdiot-Andy/pdiot-practical/pdiot-data/2022"

for sNumber in os.listdir(folder_path):
    
    for filename in os.listdir(folder_path +'/'+sNumber):
        if "Respeck" in filename:
            full_path = f"{folder_path}/{sNumber}/{filename}"
            
            new_df = pd.read_csv(full_path)
        
            base_df = pd.concat([base_df, new_df])
            

In [3]:
base_df.reset_index(drop=True, inplace=True)
base_df

,timestamp,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,sensor_type,activity_type,activity_code,subject_id,notes,recording_id
0,1.664034e+12,-0.342529,0.734070,-0.227600,-36.125000,-22.734375,1.531250,Respeck,Running,11,s1910268,10,Respeck_s1910268_Running_24-09-2022_16-43-26
1,1.664034e+12,0.399414,0.030212,-0.037903,9.156250,-11.875000,17.062500,Respeck,Running,11,s1910268,10,Respeck_s1910268_Running_24-09-2022_16-43-26
2,1.664034e+12,-0.339111,-0.026672,0.140076,23.984375,10.750000,17.171875,Respeck,Running,11,s1910268,10,Respeck_s1910268_Running_24-09-2022_16-43-26
3,1.664034e+12,-0.458984,-1.072083,-0.853577,8.171875,-4.093750,-3.500000,Respeck,Running,11,s1910268,10,Respeck_s1910268_Running_24-09-2022_16-43-26
4,1.664034e+12,1.999939,-2.000000,0.497498,67.984375,-231.250000,-10.546875,Respeck,Running,11,s1910268,10,Respeck_s1910268_Running_24-09-2022_16-43-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...
832639,1.664966e+12,0.271484,-0.057922,-0.944153,-0.515625,-0.109375,-0.031250,Respeck,Lying down on stomach,8,s2308859,lying,Respeck_s2308859_Lying down on stomach_05-10-2...
832640,1.664966e+12,0.271729,-0.061829,-0.946594,-0.468750,-0.187500,0.062500,Respeck,Lying down on stomach,8,s2308859,lying,Respeck_s2308859_Lying down on stomach_05-10-2...
832641,1.664966e+12,0.273926,-0.058411,-0.941711,-0.437500,-0.078125,-0.062500,Respeck,Lying down on stomach,8,s2308859,lying,Respeck_s2308859_Lying down on stomach_05-10-2...
832642,1.664966e+12,0.273193,-0.055725,-0.942200,-0.468750,-0.218750,0.156250,Respeck,Lying down on stomach,8,s2308859,lying,Respeck_s2308859_Lying down on stomach_05-10-2...


In [4]:
print(f"The data was collected using the sensors: {base_df.sensor_type.unique()}")
print(f"The data was collected for the activities: {base_df.activity_type.unique()}")
print(f"The number of unique recordings is: {len(base_df.recording_id.unique())}")
print(f"The subject IDs in the recordings are: {len(base_df.subject_id.unique())}")

The data was collected using the sensors: ['Respeck' 'Thingy']
The data was collected for the activities: ['Running' 'Descending stairs' 'Desk work' 'Lying down left'
 'Sitting bent backward' 'Sitting bent forward' 'Walking at normal speed'
 'Lying down on stomach' 'Lying down right' 'Sitting' 'Standing'
 'Movement' 'Lying down on back' 'Climbing stairs']
The number of unique recordings is: 1092
The subject IDs in the recordings are: 79


## Creating sliding windows

In [5]:
columns_of_interest = ['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']

In [6]:
window_size = 50 # 50 datapoints for the window size, which, at 25Hz, means 2 seconds
step_size = 25 # this is 50% overlap

window_number = 0 # start a counter at 0 to keep track of the window number

all_overlapping_windows = []

for rid, group in base_df.groupby("recording_id"):
    print(f"Processing rid = {rid}")
    
    large_enough_windows = [window for window in group.rolling(window=window_size, min_periods=window_size) if len(window) == window_size]
    
    overlapping_windows = large_enough_windows[::step_size] 
    
    # then we will append a window ID to each window
    for window in overlapping_windows:
        window.loc[:, 'window_id'] = window_number
        window_number += 1
    
    
    all_overlapping_windows.append(pd.concat(overlapping_windows).reset_index(drop=True))

Processing rid = C:\2022\iot\our_branch\Labs\Data\2022_A\day1\Respeck_s1810150_Standing_06-10-2022_20-49-00
Processing rid = Respeck_s1732873_Climbing stairs_04-10-2022_14-29-16
Processing rid = Respeck_s1732873_Descending stairs_04-10-2022_14-29-57
Processing rid = Respeck_s1732873_Desk work_05-10-2022_09-10-44
Processing rid = Respeck_s1732873_Lying down left_04-10-2022_14-12-42
Processing rid = Respeck_s1732873_Lying down on back_04-10-2022_14-11-56
Processing rid = Respeck_s1732873_Lying down on stomach_04-10-2022_14-14-08
Processing rid = Respeck_s1732873_Lying down right_04-10-2022_14-13-24
Processing rid = Respeck_s1732873_Movement_05-10-2022_10-08-43
Processing rid = Respeck_s1732873_Running_04-10-2022_14-32-19
Processing rid = Respeck_s1732873_Sitting bent backward_04-10-2022_14-09-25
Processing rid = Respeck_s1732873_Sitting bent forward_04-10-2022_14-08-40
Processing rid = Respeck_s1732873_Sitting_04-10-2022_14-07-59
Processing rid = Respeck_s1732873_Standing_04-10-2022_14-1

Processing rid = Respeck_s1832137_Lying down right_10-10-2022_17-00-00
Processing rid = Respeck_s1832137_Movement_10-10-2022_17-04-40
Processing rid = Respeck_s1832137_Running_10-10-2022_17-15-45
Processing rid = Respeck_s1832137_Sitting bent backward_10-10-2022_16-40-01
Processing rid = Respeck_s1832137_Sitting bent forward_10-10-2022_16-38-35
Processing rid = Respeck_s1832137_Sitting_10-10-2022_16-37-50
Processing rid = Respeck_s1832137_Standing_10-10-2022_16-55-55
Processing rid = Respeck_s1832137_Walking at normal speed_10-10-2022_17-12-43
Processing rid = Respeck_s1832512_Climbing stairs_05-10-2022_13-42-13
Processing rid = Respeck_s1832512_Descending stairs_05-10-2022_13-42-59
Processing rid = Respeck_s1832512_Desk work_02-10-2022_15-29-57
Processing rid = Respeck_s1832512_Lying down left_05-10-2022_11-39-41
Processing rid = Respeck_s1832512_Lying down on back_02-10-2022_15-41-05
Processing rid = Respeck_s1832512_Lying down on stomach_02-10-2022_15-44-18
Processing rid = Respeck_

Processing rid = Respeck_s1858629_Walking at normal speed_07-10-2022_08-46-49
Processing rid = Respeck_s1862323_Climbing stairs_02-10-2022_00-05-31
Processing rid = Respeck_s1862323_Descending stairs_02-10-2022_00-13-57
Processing rid = Respeck_s1862323_Desk work_01-10-2022_23-14-04
Processing rid = Respeck_s1862323_Lying down left_01-10-2022_22-45-29
Processing rid = Respeck_s1862323_Lying down on back_01-10-2022_22-39-10
Processing rid = Respeck_s1862323_Lying down on stomach_01-10-2022_23-05-46
Processing rid = Respeck_s1862323_Lying down right_01-10-2022_22-54-56
Processing rid = Respeck_s1862323_Movement_01-10-2022_23-17-54
Processing rid = Respeck_s1862323_Running_01-10-2022_23-59-27
Processing rid = Respeck_s1862323_Sitting bent backward_01-10-2022_22-25-45
Processing rid = Respeck_s1862323_Sitting bent forward_01-10-2022_22-03-29
Processing rid = Respeck_s1862323_Sitting_01-10-2022_21-57-58
Processing rid = Respeck_s1862323_Standing_01-10-2022_22-30-55
Processing rid = Respeck_

Processing rid = Respeck_s1909083_Lying down on stomach_30-09-2022_02-07-45
Processing rid = Respeck_s1909083_Lying down right_30-09-2022_02-06-48
Processing rid = Respeck_s1909083_Movement_30-09-2022_03-24-27
Processing rid = Respeck_s1909083_Running_30-09-2022_02-18-24
Processing rid = Respeck_s1909083_Sitting bent backward_30-09-2022_01-52-16
Processing rid = Respeck_s1909083_Sitting bent forward_30-09-2022_01-39-13
Processing rid = Respeck_s1909083_Sitting_30-09-2022_01-51-02
Processing rid = Respeck_s1909083_Standing_30-09-2022_01-57-02
Processing rid = Respeck_s1909083_Walking at normal speed_30-09-2022_02-13-45
Processing rid = Respeck_s1910268_Climbing stairs_03-10-2022_18-12-57
Processing rid = Respeck_s1910268_Descending stairs_03-10-2022_18-11-42
Processing rid = Respeck_s1910268_Desk work_24-09-2022_16-57-34
Processing rid = Respeck_s1910268_Lying down left_24-09-2022_16-30-49
Processing rid = Respeck_s1910268_Lying down on back_24-09-2022_16-29-08
Processing rid = Respeck_

Processing rid = Respeck_s1918258_Walking at normal speed_28-09-2022_16-03-29
Processing rid = Respeck_s1918275_Climbing stairs_05-10-2022_17-00-41
Processing rid = Respeck_s1918275_Descending stairs_05-10-2022_16-59-52
Processing rid = Respeck_s1918275_Desk work_06-10-2022_09-01-20
Processing rid = Respeck_s1918275_Lying down left_01-10-2022_13-47-27
Processing rid = Respeck_s1918275_Lying down on back_01-10-2022_13-43-54
Processing rid = Respeck_s1918275_Lying down on stomach_01-10-2022_13-50-24
Processing rid = Respeck_s1918275_Lying down right_01-10-2022_13-48-16
Processing rid = Respeck_s1918275_Movement_01-10-2022_14-11-39
Processing rid = Respeck_s1918275_Running_01-10-2022_13-56-24
Processing rid = Respeck_s1918275_Sitting bent backward_01-10-2022_13-35-58
Processing rid = Respeck_s1918275_Sitting bent forward_01-10-2022_13-34-35
Processing rid = Respeck_s1918275_Sitting_04-10-2022_14-16-41
Processing rid = Respeck_s1918275_Standing_01-10-2022_13-40-11
Processing rid = Respeck_

Processing rid = Respeck_s1925715_Walking at normal speed_28-09-2022_14-26-03
Processing rid = Respeck_s1926989_Climbing stairs_03-10-2022_17-15-49
Processing rid = Respeck_s1926989_Descending stairs_03-10-2022_17-10-22
Processing rid = Respeck_s1926989_Desk work_03-10-2022_17-08-04
Processing rid = Respeck_s1926989_Lying down left_03-10-2022_17-04-42
Processing rid = Respeck_s1926989_Lying down on back_03-10-2022_17-04-02
Processing rid = Respeck_s1926989_Lying down on stomach_03-10-2022_17-06-19
Processing rid = Respeck_s1926989_Lying down right_03-10-2022_17-05-36
Processing rid = Respeck_s1926989_Movement_03-10-2022_17-07-08
Processing rid = Respeck_s1926989_Running_03-10-2022_17-13-11
Processing rid = Respeck_s1926989_Sitting bent backward_03-10-2022_17-02-32
Processing rid = Respeck_s1926989_Sitting bent forward_03-10-2022_17-01-54
Processing rid = Respeck_s1926989_Sitting_03-10-2022_17-01-17
Processing rid = Respeck_s1926989_Standing_03-10-2022_17-03-12
Processing rid = Respeck_

Processing rid = Respeck_s1951735_Lying down right_03-10-2022_02-16-50
Processing rid = Respeck_s1951735_Movement_03-10-2022_13-27-29
Processing rid = Respeck_s1951735_Running_03-10-2022_13-58-49
Processing rid = Respeck_s1951735_Sitting bent backward_03-10-2022_00-55-12
Processing rid = Respeck_s1951735_Sitting bent forward_03-10-2022_00-44-38
Processing rid = Respeck_s1951735_Sitting_03-10-2022_00-40-18
Processing rid = Respeck_s1951735_Standing_03-10-2022_00-57-24
Processing rid = Respeck_s1951735_Walking at normal speed_03-10-2022_14-46-41
Processing rid = Respeck_s1956488_Climbing stairs_02-10-2022_15-38-43
Processing rid = Respeck_s1956488_Descending stairs_02-10-2022_15-39-30
Processing rid = Respeck_s1956488_Desk work_02-10-2022_15-41-47
Processing rid = Respeck_s1956488_Lying down left_02-10-2022_15-36-25
Processing rid = Respeck_s1956488_Lying down on back_02-10-2022_15-34-42
Processing rid = Respeck_s1956488_Lying down on stomach_06-10-2022_14-44-39
Processing rid = Respeck_

Processing rid = Respeck_s1967087_Standing_02-10-2022_15-10-17
Processing rid = Respeck_s1967087_Walking at normal speed_02-10-2022_17-11-11
Processing rid = Respeck_s1967984_Climbing stairs_06-10-2022_14-31-09
Processing rid = Respeck_s1967984_Descending stairs_06-10-2022_14-32-33
Processing rid = Respeck_s1967984_Desk work_03-10-2022_21-49-02
Processing rid = Respeck_s1967984_Lying down left_24-09-2022_16-16-46
Processing rid = Respeck_s1967984_Lying down on back_24-09-2022_16-16-02
Processing rid = Respeck_s1967984_Lying down on stomach_24-09-2022_16-18-19
Processing rid = Respeck_s1967984_Lying down right_24-09-2022_16-17-31
Processing rid = Respeck_s1967984_Movement_24-09-2022_16-27-45
Processing rid = Respeck_s1967984_Running_06-10-2022_14-49-02
Processing rid = Respeck_s1967984_Sitting bent backward_24-09-2022_16-14-07
Processing rid = Respeck_s1967984_Sitting bent forward_24-09-2022_16-13-07
Processing rid = Respeck_s1967984_Sitting_03-10-2022_21-07-36
Processing rid = Respeck_

Processing rid = Respeck_s1998909_Movement_03-10-2022_16-42-38
Processing rid = Respeck_s1998909_Running_03-10-2022_16-34-44
Processing rid = Respeck_s1998909_Sitting bent backward_03-10-2022_16-06-53
Processing rid = Respeck_s1998909_Sitting bent forward_03-10-2022_16-04-13
Processing rid = Respeck_s1998909_Sitting_03-10-2022_16-02-04
Processing rid = Respeck_s1998909_Standing_03-10-2022_16-10-30
Processing rid = Respeck_s1998909_Walking at normal speed_03-10-2022_16-32-15
Processing rid = Respeck_s2255740_Climbing stairs_01-10-2022_18-54-19
Processing rid = Respeck_s2255740_Descending stairs_01-10-2022_18-55-02
Processing rid = Respeck_s2255740_Desk work_01-10-2022_20-27-58
Processing rid = Respeck_s2255740_Lying down left_01-10-2022_21-05-40
Processing rid = Respeck_s2255740_Lying down on back_01-10-2022_20-57-06
Processing rid = Respeck_s2255740_Lying down on stomach_01-10-2022_21-15-46
Processing rid = Respeck_s2255740_Lying down right_01-10-2022_21-09-13
Processing rid = Respeck_

Processing rid = Respeck_s2342658_Walking at normal speed_04-10-2022_14-53-01
Processing rid = Stairs\Respeck_s1834237_Climbing stairs_30-09-2022_17-56-16
Processing rid = Stairs\Respeck_s1834237_Descending stairs_30-09-2022_17-57-05
Processing rid = files\Respeck_s1834237_Desk work_30-09-2022_17-49-52
Processing rid = files\Respeck_s1834237_Lying down left_30-09-2022_17-14-37
Processing rid = files\Respeck_s1834237_Lying down on back_30-09-2022_17-09-25
Processing rid = files\Respeck_s1834237_Lying down on stomach_30-09-2022_17-17-45
Processing rid = files\Respeck_s1834237_Lying down right_30-09-2022_17-12-08
Processing rid = files\Respeck_s1834237_Movement_30-09-2022_17-59-29
Processing rid = files\Respeck_s1834237_Running_30-09-2022_17-44-40
Processing rid = files\Respeck_s1834237_Sitting bent backward_30-09-2022_16-59-23
Processing rid = files\Respeck_s1834237_Sitting bent forward_30-09-2022_16-58-23
Processing rid = files\Respeck_s1834237_Sitting_30-09-2022_16-55-17
Processing rid

In [7]:
final_sliding_windows = pd.concat(all_overlapping_windows).reset_index(drop=True)


In [8]:
# now extract all features
feature_list = []

for col in columns_of_interest:
    new_features = tsfresh.extract_features(timeseries_container=final_sliding_windows, column_id='window_id',
                        column_value=col, default_fc_parameters=tsfresh.feature_extraction.MinimalFCParameters())
    
    feature_list.append(new_features)
    
feature_list = pd.concat(feature_list, axis=1)

feature_list

ValueError: Column must not contain NaN values: accel_x

## Adding labels

In [52]:
dp_act = final_sliding_windows.groupby("window_id")[['activity_type']].agg(np.min)

In [54]:
np_acts = np.unique(dp_act.to_numpy())

In [57]:
class_labels = {}

acts_num = 0
for acts in np_acts:
    class_labels[acts] = acts_num
    acts_num = acts_num + 1

In [58]:
class_labels

{'Climbing stairs': 0,
 'Descending stairs': 1,
 'Desk work': 2,
 'Lying down left': 3,
 'Lying down on back': 4,
 'Lying down on stomach': 5,
 'Lying down right': 6,
 'Movement': 7,
 'Running': 8,
 'Sitting': 9,
 'Sitting bent backward': 10,
 'Sitting bent forward': 11,
 'Standing': 12,
 'Walking at normal speed': 13}

In [60]:
window_id_class_labels = final_sliding_windows.groupby("window_id")[['activity_type']].agg(np.min).replace(class_labels)
window_id_class_labels

,activity_type
window_id,
0,0
1,0
2,0
3,0
4,0
...,...
399,13
400,13
401,13


# Training a simple 1d CNN

In [78]:
filters = 64
kernel_size = 3
n_features = 6
activation='relu'
n_classes = 14

In [79]:
model = Sequential()

model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='linear', 
                 input_shape=(window_size, n_features)))
model.add(BatchNormalization())
model.add(Activation(activation))

model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='linear'))
model.add(BatchNormalization())
model.add(Activation(activation))

model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='linear'))
model.add(BatchNormalization())
model.add(Activation(activation))

model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 48, 64)            1216      
                                                                 
 batch_normalization_3 (Batc  (None, 48, 64)           256       
 hNormalization)                                                 
                                                                 
 activation_3 (Activation)   (None, 48, 64)            0         
                                                                 
 conv1d_4 (Conv1D)           (None, 46, 64)            12352     
                                                                 
 batch_normalization_4 (Batc  (None, 46, 64)           256       
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (None, 46, 64)           

In [80]:
X = []
y = []

for window_id, group in final_sliding_windows.groupby('window_id'):
    print(f"window_id = {window_id}")
    
    shape = group[columns_of_interest].values.shape
    print(f"shape = {shape}")
    
    X.append(group[columns_of_interest].values)
    y.append(class_labels[group["activity_type"].values[0]])

window_id = 0
shape = (50, 6)
window_id = 1
shape = (50, 6)
window_id = 2
shape = (50, 6)
window_id = 3
shape = (50, 6)
window_id = 4
shape = (50, 6)
window_id = 5
shape = (50, 6)
window_id = 6
shape = (50, 6)
window_id = 7
shape = (50, 6)
window_id = 8
shape = (50, 6)
window_id = 9
shape = (50, 6)
window_id = 10
shape = (50, 6)
window_id = 11
shape = (50, 6)
window_id = 12
shape = (50, 6)
window_id = 13
shape = (50, 6)
window_id = 14
shape = (50, 6)
window_id = 15
shape = (50, 6)
window_id = 16
shape = (50, 6)
window_id = 17
shape = (50, 6)
window_id = 18
shape = (50, 6)
window_id = 19
shape = (50, 6)
window_id = 20
shape = (50, 6)
window_id = 21
shape = (50, 6)
window_id = 22
shape = (50, 6)
window_id = 23
shape = (50, 6)
window_id = 24
shape = (50, 6)
window_id = 25
shape = (50, 6)
window_id = 26
shape = (50, 6)
window_id = 27
shape = (50, 6)
window_id = 28
shape = (50, 6)
window_id = 29
shape = (50, 6)
window_id = 30
shape = (50, 6)
window_id = 31
shape = (50, 6)
window_id = 32
sha

In [81]:
X = np.asarray(X)
y = np.asarray(y)

print(f"X shape = {X.shape}")
print(f"y shape = {y.shape}")

X shape = (404, 50, 6)
y shape = (404,)


In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, train_size=0.8)

y_train = np.asarray(pd.get_dummies(y_train), dtype=np.float32)
y_test = np.asarray(pd.get_dummies(y_test), dtype=np.float32)

In [83]:
print(f"X_train shape = {X_train.shape}")
print(f"y_train shape = {y_train.shape}")

print(f"X_test shape = {X_test.shape}")
print(f"y_test shape = {y_test.shape}")

X_train shape = (323, 50, 6)
y_train shape = (323, 14)
X_test shape = (81, 50, 6)
y_test shape = (81, 14)


In [84]:
model.compile(
    optimizer=optimizers.SGD(lr=0.001),
    loss='binary_crossentropy',
    metrics = ['accuracy'])

/home/jingyuan/.local/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [85]:
model.fit(X_train, y_train,
        batch_size=2, epochs=10)

Epoch 1/10
162/162 [==============================] - 1s 4ms/step - loss: 0.6167 - accuracy: 0.0774
Epoch 2/10
162/162 [==============================] - 1s 4ms/step - loss: 0.4503 - accuracy: 0.0588
Epoch 3/10
162/162 [==============================] - 1s 4ms/step - loss: 0.3656 - accuracy: 0.0650
Epoch 4/10
162/162 [==============================] - 1s 4ms/step - loss: 0.3216 - accuracy: 0.0960
Epoch 5/10
162/162 [==============================] - 1s 4ms/step - loss: 0.2932 - accuracy: 0.1084
Epoch 6/10
162/162 [==============================] - 1s 4ms/step - loss: 0.2810 - accuracy: 0.0898
Epoch 7/10
162/162 [==============================] - 1s 5ms/step - loss: 0.2701 - accuracy: 0.1176
Epoch 8/10
162/162 [==============================] - 1s 4ms/step - loss: 0.2621 - accuracy: 0.1486
Epoch 9/10
162/162 [==============================] - 1s 4ms/step - loss: 0.2613 - accuracy: 0.1393
Epoch 10/10
162/162 [==============================] - 1s 4ms/step - loss: 0.2517 - accuracy: 0.1765

In [86]:
# stats
y_pred_ohe = model.predict(X_test)
y_pred_labels = np.argmax(y_pred_ohe, axis=1)
y_true_labels = np.argmax(y_test, axis=1)

3/3 [==============================] - 0s 3ms/step


In [87]:
print("*" * 80)
print("Classification report")
print("*" * 80)
print(classification_report(y_true_labels, y_pred_labels))

********************************************************************************
Classification report
********************************************************************************
              precision    recall  f1-score   support

           0       0.33      0.20      0.25         5
           1       0.33      0.40      0.36         5
           2       0.00      0.00      0.00         6
           3       0.00      0.00      0.00         9
           4       0.00      0.00      0.00         5
           5       0.00      0.00      0.00         4
           6       0.00      0.00      0.00         4
           7       0.20      0.17      0.18         6
           8       1.00      0.25      0.40         8
           9       0.00      0.00      0.00         8
          10       0.08      1.00      0.15         4
          11       0.00      0.00      0.00         4
          12       0.00      0.00      0.00         5
          13       0.00      0.00      0.00         8

    

/home/jingyuan/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jingyuan/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jingyuan/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
